In [1]:
import scipy.io as sio
from scipy.misc import imread
import tensorflow as tf
import numpy as np
import pandas as pd
import sys
import os
import scipy.io
from find_largest_image import find_largest
import tqdm 
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from xgboost import XGBClassifier
import random
from numpy.random import choice
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, roc_curve
import sklearn.pipeline as pipeline
import sklearn.preprocessing as preprocessing
from sklearn.model_selection import train_test_split

In [2]:
data_folder = 'C:\\Users\\Mert\\Documents\\GitHub\\sigver_bmg\\data\\GPDSSyntheticSignatures4k\\bmgnet-f0999_features'

In [3]:
data_f = pd.read_csv(os.path.join(data_folder,'data_features.csv'))
visual_f = pd.read_csv(os.path.join(data_folder,'visual_features.csv'))

# MODEL SELECTION & TRAINING

In [4]:
dev_exp_ratio = 0.3
sorted_id_list = np.sort(data_f['user_id'].unique())
dev_exp_splitter=int(len(sorted_id_list)*dev_exp_ratio)
dev_val_user_ids = sorted_id_list[:dev_exp_splitter]
exp_user_ids = sorted_id_list[dev_exp_splitter:]

In [5]:
dev_val_user_ids.shape

(1200,)

In [6]:
exp_user_ids.shape

(2800,)

In [7]:
fakes_preds = []
gens_preds = []

for fold in np.arange(0,1):
    # assert len(dev_val_user_ids)==581
    np.random.shuffle(dev_val_user_ids)
    dev_user_ids = dev_val_user_ids[0:1000]
    validation_user_ids = dev_val_user_ids[1000:len(dev_val_user_ids)]
    train_idx, test_idx = train_test_split(np.arange(1,25), train_size=0.5, test_size=0.5)

    dev_df = data_f.loc[data_f['user_id'].isin(dev_user_ids)]
    dev_vf = visual_f.loc[dev_df.index]
    val_df = data_f.loc[data_f['user_id'].isin(validation_user_ids)]
    val_vf = visual_f.loc[val_df.index]

    dev_df_gen = dev_df.loc[dev_df['fakeness']==0]
    dev_df_fake = dev_df.loc[dev_df['fakeness']==1]
    dev_df_gen_12 = dev_df_gen.loc[dev_df_gen['sig_id'].isin(train_idx)]
    dev_df_valid_12 = dev_df_gen.loc[dev_df_gen['sig_id'].isin(test_idx)]
    
    train_idx, test_idx = train_test_split(np.arange(1,25), train_size=0.5)
    val_df_gen = val_df.loc[val_df['fakeness']==0]
    val_df_fake = val_df.loc[val_df['fakeness']==1]
    val_df_gen_12 = val_df_gen.loc[val_df_gen['sig_id'].isin(train_idx)]
    val_df_valid_gen_12 = val_df_gen.loc[val_df_gen['sig_id'].isin(test_idx)]

    for user_id in tqdm.tqdm(validation_user_ids, ascii=True):
        clf = MLPClassifier(hidden_layer_sizes=(100,50))
        y_train = (pd.concat([val_df_gen_12.loc[val_df_gen_12['user_id']==user_id],dev_df_gen.loc[dev_df_gen['user_id']!=user_id]]))['user_id']==user_id
        X_train = visual_f.loc[y_train.index]  
        clf.fit(X_train, y_train)
        y_valid_fakes = val_df_fake.loc[(val_df_fake['user_id']==user_id)]
        X_valid_f = visual_f.loc[y_valid_fakes.index]
        fakes_preds.append(clf.predict_proba(X_valid_f)[:,1])
        y_valid_gens = val_df_valid_gen_12.loc[val_df_valid_gen_12['user_id']==user_id]
        X_valid_g = visual_f.loc[y_valid_gens.index]
        gens_preds.append(clf.predict_proba(X_valid_g)[:,1])

c:\users\mert\anaconda3\envs\ml_gpu\lib\site-packages\sklearn\model_selection\_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)
100%|##############################################################################| 200/200 [1:19:23<00:00, 20.51s/it]


# GLOBAL THRESHOLD SELECTION

In [8]:
flat_fakes_preds = np.expand_dims(np.array([item for sublist in fakes_preds for item in sublist]),axis=1)
flat_gens_preds = np.expand_dims(np.array([item for sublist in gens_preds for item in sublist]),axis=1)
all_preds = np.vstack((flat_fakes_preds,flat_gens_preds))
all_labels = np.vstack((np.zeros((flat_fakes_preds.shape[0],1)),np.ones((flat_gens_preds.shape[0],1))))

fpr,tpr,threshold = roc_curve(all_labels,all_preds)
fnr = 1 - tpr
EER = fpr[np.nanargmin(np.absolute((fnr - fpr)))]
eer_th = threshold[np.nanargmin(np.absolute((fnr - fpr)))]
print('EER_glob : ', EER*100,'\nEER_Threshold_glob : ', eer_th)
glob_th = eer_th

EER_glob :  7.533333333333333 
EER_Threshold_glob :  0.04184683638722593


In [9]:
assert len(fakes_preds)==len(gens_preds)
EER_accum=0
for idx,val in enumerate(fakes_preds):
    user_fakes_preds = np.expand_dims(np.array(fakes_preds[idx]),axis=1)
    user_gens_preds = np.expand_dims(np.array(gens_preds[idx]),axis=1)
    all_user_preds = np.vstack((user_fakes_preds,user_gens_preds))
    all_user_labels = np.vstack((np.zeros((user_fakes_preds.shape[0],1)),np.ones((user_gens_preds.shape[0],1)))) 
    fpr,tpr,threshold = roc_curve(all_user_labels,all_user_preds)
    fnr = 1 - tpr
    EER = fpr[np.nanargmin(np.absolute((fnr - fpr)))]
    EER_accum += EER
    
print('EER_user : ', (EER_accum*100)/len(fakes_preds)) 

EER_user :  4.616666666666662


In [10]:
print(glob_th)

0.04184683638722593


# TRAIN AND TEST ON THE EXPLOITATION SET

In [11]:
test_gens_preds = []
test_fakes_preds = []

train_idx, test_idx = train_test_split(np.arange(1,25), train_size=0.5)
exp_df = data_f.loc[data_f['user_id'].isin(exp_user_ids)]
exp_vf = visual_f.loc[exp_df.index]
exp_df_gen = exp_df.loc[exp_df['fakeness']==0]
exp_df_fake = exp_df.loc[exp_df['fakeness']==1]
exp_df_fake_10 = exp_df_fake.loc[exp_df_fake['sig_id'].isin(choice(np.arange(1,31),10,replace=False))]
exp_df_gen_12 = exp_df_gen.loc[exp_df_gen['sig_id'].isin(train_idx)]
exp_df_valid_gen_10 = exp_df_gen.loc[exp_df_gen['sig_id'].isin(test_idx[0:10])]

dev_val_df = data_f.loc[data_f['user_id'].isin(dev_val_user_ids)]
dev_val_vf = visual_f.loc[dev_val_df.index]
dev_val_df_gen = dev_val_df.loc[dev_val_df['fakeness']==0]
dev_val_df_valid_gen_14 = dev_val_df_gen.loc[dev_val_df_gen['sig_id'].isin(choice(np.arange(1,25),14,replace=False))]

c:\users\mert\anaconda3\envs\ml_gpu\lib\site-packages\sklearn\model_selection\_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


MemoryError: 

In [ ]:
for user_id in tqdm.tqdm(exp_user_ids, ascii=True):
    clf = SVC(C=1,gamma='scale',class_weight='balanced', probability=False, kernel=user_kernel)
    y_train = (pd.concat([exp_df_gen_12.loc[exp_df_gen_12['user_id']==user_id],dev_val_df_valid_gen_14.loc[dev_val_df_valid_gen_14['user_id']!=user_id]]))['user_id']==user_id
    X_train = visual_f.loc[y_train.index]  
    clf.fit(X_train, y_train)
    y_valid_fakes = exp_df_fake_10.loc[(exp_df_fake_10['user_id']==user_id)]
    X_valid_f = visual_f.loc[y_valid_fakes.index]
    test_fakes_preds.append(clf.decision_function(X_valid_f))
    y_valid_gens = exp_df_valid_gen_10.loc[exp_df_valid_gen_10['user_id']==user_id]
    X_valid_g = visual_f.loc[y_valid_gens.index]
    test_gens_preds.append(clf.decision_function(X_valid_g))

In [ ]:
flat_test_fakes_preds = np.expand_dims(np.array([item for sublist in test_fakes_preds for item in sublist]),axis=1)
flat_test_gens_preds = np.expand_dims(np.array([item for sublist in test_gens_preds for item in sublist]),axis=1)
print("____At the EER threshold decided on the Validation set____")
print("FRR : ",(1-len(flat_test_gens_preds[flat_test_gens_preds>=glob_th])/len(flat_test_gens_preds))*100)
print("FARskilled : ",(1-len(flat_test_fakes_preds[flat_test_fakes_preds<glob_th])/len(flat_test_fakes_preds))*100)

In [ ]:
all_test_preds = np.vstack((flat_test_fakes_preds,flat_test_gens_preds))
all_test_labels = np.vstack((np.zeros((flat_test_fakes_preds.shape[0],1)),np.ones((flat_test_gens_preds.shape[0],1))))

fpr,tpr,threshold = roc_curve(all_test_labels,all_test_preds)
fnr = 1 - tpr
EER = fpr[np.nanargmin(np.absolute((fnr - fpr)))]
eer_th = threshold[np.nanargmin(np.absolute((fnr - fpr)))]
print('EER_glob for test set: ', EER*100,'\nEER_Threshold_glob for test set: ', eer_th)

In [ ]:
assert len(test_fakes_preds)==len(test_gens_preds)
EER_accum=0
for idx,val in enumerate(test_fakes_preds):
    user_test_fakes_preds = np.expand_dims(np.array(test_fakes_preds[idx]),axis=1)
    user_test_gens_preds = np.expand_dims(np.array(test_gens_preds[idx]),axis=1)
    all_user_test_preds = np.vstack((user_test_fakes_preds,user_test_gens_preds))
    all_user_test_labels = np.vstack((np.zeros((user_test_fakes_preds.shape[0],1)),np.ones((user_test_gens_preds.shape[0],1)))) 
    fpr,tpr,threshold = roc_curve(all_user_test_labels,all_user_test_preds)
    fnr = 1 - tpr
    EER = fpr[np.nanargmin(np.absolute((fnr - fpr)))]
    EER_accum += EER
    
print('EER_user for test set : ', (EER_accum*100)/len(test_fakes_preds))